In [1]:
from collections import namedtuple
import os
os.chdir(r"C:\Users\Praveen\deepCNNClassifier\deepCNNClassifier") # making the parent directory as jupyter notebook consuder the research folder
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories
import tensorflow as tf


In [2]:
!pip install scipyS

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augumentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self)-> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks  # all values that are defined configs.yaml availabe here
        model_check_point_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(model_check_point_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config

    def get_training_config(self)-> TrainingConfig:
        training = self.config.training  # all values that are defined configs.yaml availabe here
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")

        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augumentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )
        return training_config

In [5]:
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir) # it expects log directori path
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True # it will save the best weights only
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,  # defined it as a property
            self._create_ckpt_callbacks # defined it as a property
        ]

In [6]:
# trainingComponent
import os
import urllib.request as request
from zipfile import ZipFile
#import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs= dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],# no need to pass channel
            batch_size= self.config.params_batch_size,
            interpolation="bilinear" # it is used to resizing the images
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augumentation:
            training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2, # applying some type of pressure, so it will tilt the image
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            training_datagenerator=valid_datagenerator

        self.train_generator = training_datagenerator.flow_from_directory(
            self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    def train(self, callback_list:list):
        self.steps_per_epoch=self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.train_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model

        )
    

In [7]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callback_list=prepare_callback.get_tb_ckpt_callbacks()
    
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list = callback_list
    )
except Exception as e:
    raise e
#here 20000/16

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
  46/1250 [>.............................] - ETA: 2:22:56 - loss: 16.5293 - accuracy: 0.5054

In [ ]:
def example(x, **kwargs):
    print(locals())

In [ ]:
example(x=3, y=4, z=0)

{'x': 3, 'kwargs': {'y': 4, 'z': 0}}


In [ ]:
# same defined it as like below
extra= dict(a=4,b=3)

example(x=3, **extra)

{'x': 3, 'kwargs': {'a': 4, 'b': 3}}
